### Conditional Workflow with Microsoft Agent Framework

![conditional_workflow](./images/conditional_workflow.png)

In [ ]:
%pip install agent-framework==1.0.0b251001 python-dotenv azure-ai-projects==1.1.0b4

### Import Required Dependencies

In [ ]:
import asyncio
import os
from dataclasses import dataclass
from typing import Any, Literal
from uuid import uuid4

from typing_extensions import Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    Role,
    WorkflowBuilder,
    WorkflowContext,
    executor,
    Case,
    Default,
)
from agent_framework.azure import AzureOpenAIChatClient
from azure.identity import AzureCliCredential
from pydantic import BaseModel

### Setting Up the Environment

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("AZURE_OPENAI_API_KEY")
endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
deployment_name = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")

print(f"API Key: {api_key}")
print(f"Endpoint: {endpoint}")
print(f"Deployment Name: {deployment_name}")


### Creating AzureOpenAIChatClient

In [ ]:
from agent_framework.azure import AzureOpenAIChatClient

# Creating the AzureOpenAIChatClient
client = AzureOpenAIChatClient(
    api_key = api_key,
    endpoint = endpoint,
    deployment_name = deployment_name
)


### Defining the Data Models

In [ ]:
from pydantic import BaseModel

class RoutingResult(BaseModel):
    is_technical_question: bool
    original_question: str

class AgentResponse(BaseModel):
    answer: str

### Creating Router Agent

In [ ]:
# Setting the system prompt for the Router Agent
system_prompt_router_agent = """
You are the Interview Router Agent.

Your task:
1. Read the user’s question carefully.
2. Decide whether it belongs to:
   - "CodingAgent" → if it involves technical topics such as programming, data structures, algorithms, time complexity, debugging, or software design.
   - "BehavioralAgent" → if it involves personality, teamwork, motivation, HR, or experience-based questions.

Examples:
Q: "Explain quicksort." → coding
Q: "Tell me about a time you handled conflict." → behavioral
Q: "How would you optimize an algorithm?" → coding
Q: "What motivates you?" → behavioral

Output Instructions:
Always return JSON with fields `is_technical_question` set to true or false and `original_question` containing the user's question.
"""

# Creating the Router Agent
router_agent = AgentExecutor(client.create_agent(
    instructions = system_prompt_router_agent,
    response_format = RoutingResult,
),
    id = "RouterAgent"
)

### Creating the Coding Agent (The Technical Interviewer Coach)

In [ ]:
# Setting the system prompt for the Coding Agent (The Technical Interviewer Coach)
system_prompt_coding_agent = """
You are the Coding Interview Coach Agent.

Your job is to help users understand and solve programming and algorithmic interview questions.

Guidelines:
- Explain concepts clearly and step-by-step.
- When appropriate, include small, readable code examples in Python.
- Mention time and space complexity.
- Use simple language — imagine you are teaching a student preparing for FAANG interviews.
- End your response with one practical takeaway (e.g., “Try implementing this recursively and iteratively.”).

Tone:
- Friendly, confident, and encouraging — like a mentor.

Examples:
Q: "What is binary search?"
A: Binary search is a divide-and-conquer algorithm that runs in O(log n) time...

"""

# Creating the Coding Agent (The Technical Interviewer Coach)
coding_agent = AgentExecutor(client.create_agent(
    instructions = system_prompt_coding_agent,
    response_format = AgentResponse,
),
    id = "CodingAgent"
)

### Creating the Behavioral Agent (The HR Interviewer Coach)

In [ ]:
# Setting the system prompt for the Behavioral Agent (The HR Interviewer Coach)
system_prompt_behavioral_agent = """
You are the Behavioral Interview Coach Agent.

Your role is to help users prepare for behavioral or HR interview questions using the STAR method.

Guidelines:
- Always structure responses using the STAR framework:
  **Situation:** Describe the context.
  **Task:** What was expected of you?
  **Action:** What you did.
  **Result:** The measurable or emotional outcome.
- Keep answers concise and positive.
- Use professional but conversational tone.
- If the question is vague, help the user brainstorm a suitable story.
- End with one improvement tip (e.g., “Quantify your impact to make the story stronger.”).

Examples:
Q: "Tell me about a time you led a team."
A:
**Situation:** During my college robotics project...
**Task:** I was responsible for coordinating the design and programming...
**Action:** I divided tasks, set up weekly check-ins, and helped resolve conflicts.
**Result:** We won 2nd place at the national robotics competition.

"""

# Creating the Behavioral Agent (The HR Interviewer Coach)
behavioral_agent = AgentExecutor(client.create_agent(
    instructions = system_prompt_behavioral_agent,
    response_format = AgentResponse
),
    id = "BehavioralAgent"
)

### Creating the Condition Evaluation Function

In [ ]:
def decide_agent_routing(expected_result: bool):

    # The returned function will be used as an edge predicate.
    # It receives whatever the upstream executor produced.
    def condition(message: Any) -> bool:
        try:
            routing_result = RoutingResult.model_validate_json(message.agent_run_response.text)

            # returns true if the expected routing result is that it is a technical question
            # if its not a technical question, it returns false - which means it should go to the behavioral agent
            return routing_result.is_technical_question == expected_result
        except Exception as e:
            return False
    
    return condition

### Creating Handler Executors

In [ ]:
@executor
async def forward_request_to_coding_agent(
    response: AgentExecutorResponse,
    ctx: WorkflowContext[AgentExecutorRequest]
) -> None:
    # Parse the routing result from the Router Agent's response and extract the original question
    # the original question is then set as the input to the next agent (either coding or behavioral)
    routing_result = RoutingResult.model_validate_json(response.agent_run_response.text)

    request = AgentExecutorRequest(
        messages = [ChatMessage(
            Role.USER, text=routing_result.original_question)],
            should_respond=True
    )

    await ctx.send_message(request)

@executor
async def forward_request_to_behavioral_agent(
    response: AgentExecutorResponse,
    ctx: WorkflowContext[AgentExecutorRequest]
) -> None:
    # Parse the routing result from the Router Agent's response and extract the original question
    # the original question is then set as the input to the next agent (either coding or behavioral)
    routing_result = RoutingResult.model_validate_json(response.agent_run_response.text)

    request = AgentExecutorRequest(
        messages = [ChatMessage(
            Role.USER, text=routing_result.original_question)],
            should_respond=True
    )

    await ctx.send_message(request)

@executor
async def handle_agent_response(
    response: AgentExecutorResponse,
    ctx: WorkflowContext[AgentExecutorRequest]
) -> None:
    agent_response = AgentResponse.model_validate_json(response.agent_run_response.text)
    await ctx.yield_output("final agent answer: \n" + agent_response.answer)

### Build the Conditional Workflow

In [ ]:
workflow = (
    WorkflowBuilder()
    .set_start_executor(router_agent)

    # Route to coding path
    .add_edge(router_agent, forward_request_to_coding_agent, condition=decide_agent_routing(True))
    .add_edge(forward_request_to_coding_agent, coding_agent)
    .add_edge(coding_agent, handle_agent_response)

    # Route to behavioral path
    .add_edge(router_agent, forward_request_to_behavioral_agent, condition=decide_agent_routing(False))
    .add_edge(forward_request_to_behavioral_agent, behavioral_agent)
    .add_edge(behavioral_agent, handle_agent_response)
    .build()
)


### Visualizing the Workflow

In [ ]:
from agent_framework import WorkflowBuilder, WorkflowViz

viz = WorkflowViz(workflow)
mermaid_content = viz.to_mermaid()

# printing mermaid content as markdown
from IPython.display import Markdown, display
display(Markdown(f"```mermaid\n{mermaid_content}\n```"))

### Executing the Workflow

In [ ]:
# intentionaly using a technical question to see if the workflow routes it correctly to the Coding Agent
request = AgentExecutorRequest(
    messages = [ChatMessage(
        Role.USER, text="Can you explain the concept of polymorphism in object-oriented programming?")],
    should_respond=True
)
events = await workflow.run(request)
outputs = events.get_outputs()
if outputs:
    print(outputs[0])
else:
    print("No output received from the workflow.")

In [ ]:
# intentionaly using a behavioral question to see if the workflow routes it correctly to the Behavioral Agent
request = AgentExecutorRequest(
    messages = [ChatMessage(
        Role.USER, text="Can you tell me about a time you faced a conflict at work and how you handled it?")],
    should_respond=True
)
events = await workflow.run(request)
outputs = events.get_outputs()
if outputs:
    print(outputs[0])
else:
    print("No output received from the workflow.")

In [ ]:
request = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="Explain polymorphism")],
    should_respond=True
)

events = await workflow.run(request)
outputs = events.get_outputs()

if outputs:
    print("Workflow Output:", outputs[0])
else:
    print("No output received.")
